In [1]:
# Imports

import os
from dotenv import load_dotenv
import requests
from openai import OpenAI
from bs4 import BeautifulSoup
import ollama
from IPython.display import Markdown, display

ModuleNotFoundError: No module named 'ollama'

In [10]:
# 🔐 Load API key from .env

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"✅ OpenAI API Key found: {openai_api_key[:8]}...")
else:
    print("❌ OpenAI API Key not set!")

✅ OpenAI API Key found: sk-proj-...


In [11]:
# 🎯 Initialize OpenAI

openai = OpenAI(api_key=openai_api_key)
MODEL = "gpt-4o"

In [12]:
# 🧱 Define headers to avoid scraping blocks

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [13]:
# 🏗️ Class to represent a job listing webpage

class Job_Platform:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        # Remove unwanted tags
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator="\n", strip=True)


In [14]:
# 💬 Build a user prompt from the job listing
def user_prompt_for(job_listing):
    user_prompt = f"You are looking at a job listing titled: {job_listing.title}\n"
    user_prompt += "The contents of this listing are as follows. Please provide a short summary in markdown format. \
If there are job requirements, duties, or application instructions, highlight them:\n\n"
    user_prompt += job_listing.text
    return user_prompt


In [15]:
# 🧠 Send the user/system prompt to OpenAI
def ask_openai(system_prompt, user_prompt, model=MODEL):
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2
    )
    return response.choices[0].message.content


In [19]:
# 📂 Load resume from file
with open("resume.txt", "r") as file:
    my_resume = file.read()


In [22]:
print(my_resume[:500])  # Show the first 500 characters to make sure it loaded


 
Robin Warden
Software Engineer | Frontend & API Developer | AI Automation Enthusiast
Lawrenceville, GA       
(470) 406-1714     
Email| LinkedIn | GitHub | Portfolio

+Professional Summary

Creative and AI-native Frontend Developer with 3+ years of experience building scalable web and mobile interfaces using React, TypeScript, HTML/CSS, and modern CSS frameworks including Styled Components, Bootstrap, and Tailwind CSS. SME-level React engineer with a strong focus on component reusability, sca


In [20]:
# 📝 Define the system's behavior
system_prompt = (
    "You are a job assistant. I will give you a job listing, and you must compare it with my resume. "
    "If it's a good match, summarize the job in markdown and say why it's a fit. If it's not, reply: 'Not a match.'\n\n"
    f"My resume:\n{my_resume}"
)



In [23]:
# 🚀 Test the job finder on a real job listing
test_url = "https://remoteok.com/remote-javascript-jobs"  #Replace with the url of the job listing you want to test
# Initialize the page and build the prompt
job_page = Job_Platform(test_url)
user_prompt = user_prompt_for(job_page)

# Call OpenAI
summary = ask_openai(system_prompt, user_prompt)

# Show result
print(summary)


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-KadiOLp7HmGvhLDLEAnSxr7F on tokens per min (TPM): Limit 30000, Requested 33803. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}